# Azure AI Language Challenge Solution

## Install Library

In [ ]:
%pip install azure-ai-textanalytics

## Load Azure Configurations

In [1]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

## Create a Text Analytics client

In [2]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure Text Analytics client using the provided key and endpoint.

    Returns:
        TextAnalyticsClient: An authenticated client for Azure Text Analytics.
    """
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=ta_credential
    )
    return text_analytics_client

# Initialize the client
client = authenticate_client()

## Functions

In [3]:
def sentiment_analysis_with_opinion_mining(client, documents):
    """
    Analyzes the sentiment of the provided documents and extracts opinions using the Azure Text Analytics client.

    Args:
        client (TextAnalyticsClient): An authenticated Azure Text Analytics client.
        documents (list of str): A list of text documents to analyze for sentiment and opinions.

    Returns:
        None: Prints the sentiment analysis results, including overall sentiment, sentence-level sentiment, 
              and mined opinions for each document.
    """
    # Call the Azure Text Analytics API to analyze sentiment with opinion mining
    result = client.analyze_sentiment(documents, show_opinion_mining=True)

    # Filter out documents with errors
    doc_result = [doc for doc in result if not doc.is_error]

    # Iterate through the results for each valid document
    for doc_idx, document in enumerate(doc_result, start=1):  # Start index at 1 for readability
        # Print the overall sentiment of the document
        print("Document {} Sentiment: {}".format(doc_idx, document.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f}".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))

        # Iterate through each sentence in the document
        for idx, sentence in enumerate(document.sentences, start=1):
            # Print the sentiment of the sentence
            print("  Sentence {}: {}".format(idx, sentence.text))
            print("    Sentiment: {}".format(sentence.sentiment))
            print("    Scores: Positive={0:.2f}, Neutral={1:.2f}, Negative={2:.2f}".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))

            # Initialize a counter for mined opinions
            opinion_counter = 1

            # Iterate through mined opinions in the sentence
            for mined_opinion in sentence.mined_opinions:
                # Label the mined opinion
                print(f"    Mined Opinion {opinion_counter}:")
                opinion_counter += 1

                # Print the target of the opinion
                target = mined_opinion.target
                print("      Target '{}' ({})".format(target.text, target.sentiment))
                print("      Scores: Positive={0:.2f}, Negative={1:.2f}".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))

                # Print the assessments related to the target
                for assessment in mined_opinion.assessments:
                    print("        Assessment '{}' ({})".format(assessment.text, assessment.sentiment))
                    print("        Scores: Positive={0:.2f}, Negative={1:.2f}".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))

        # Add a blank line after each document for better readability
        print()

In [4]:
def entity_recognition(client, documents):
    """
    Recognizes named entities in the provided documents using the Azure Text Analytics client.

    Args:
        client (TextAnalyticsClient): An authenticated Azure Text Analytics client.
        documents (list of str): A list of text documents to analyze for named entities.

    Returns:
        None: Prints the recognized entities and their details for each document.
    """
    # Call the Azure Text Analytics API to recognize entities in the documents
    result = client.recognize_entities(documents=documents)

    print("Named Entities:\n")

    # Iterate through the results for each document
    for idx, doc_result in enumerate(result):  # Enumerate to get the document index
        print(f"Document {idx + 1}:\n")  # Print the document number (1-based index)

        # Check if the result for the document is valid
        if not doc_result.is_error:
            # Iterate through the recognized entities in the document
            for entity in doc_result.entities:
                # Print details of each entity
                print(
                    "\tText: \t", entity.text,
                    "\tCategory: \t", entity.category,
                    "\tSubCategory: \t", entity.subcategory,
                    "\n\tConfidence Score: \t", round(entity.confidence_score, 2),
                    "\tLength: \t", entity.length,
                    "\tOffset: \t", entity.offset,
                    "\n"
                )
        else:
            # Print an error message if the document result contains an error
            print(f"\tError in document with ID {doc_result.id}: {doc_result.error}")
        
        # Add a blank line between documents for better readability
        print("\n")

In [5]:
def extractive_summarization(client, documents):
    """
    Performs extractive summarization on the provided documents.

    Args:
        client (TextAnalyticsClient): The authenticated Azure Text Analytics client.
        documents (list): A list of documents (strings) to summarize.

    Prints:
        Extracted sentences for each document or error messages if any.
    """
    # Start the extractive summarization process
    poller = client.begin_extract_summary(
        documents,
        max_sentence_count=5  # Limit the summary to 5 sentences
    )
    extract_summary_results = poller.result()

    # Iterate through the results and print summaries or errors
    for doc_index, document in enumerate(extract_summary_results, start=1):
        if document.kind == "ExtractiveSummarization":
            print(f"Document {doc_index} Summary:")
            for i, sentence in enumerate(document.sentences, start=1):
                print(f"  Sentence {i}: {sentence.text}")
        elif document.is_error is True:
            print(f"Document {doc_index} has an error with code '{document.error.code}' and message '{document.error.message}'")

In [6]:
import textwrap

def abstractive_summarization(client, documents):
    """
    Performs abstractive summarization on the provided documents.

    Args:
        client (TextAnalyticsClient): The authenticated Azure Text Analytics client.
        documents (list): A list of documents (strings) to summarize.

    Prints:
        Abstractive summaries for each document or error messages if any.
    """
    # Start the abstractive summarization process
    poller = client.begin_abstract_summary(documents)
    abstract_summary_results = poller.result()

    # Iterate through the results and print summaries or errors
    for doc_index, result in enumerate(abstract_summary_results, start=1):
        if result.kind == "AbstractiveSummarization":
            print(f"Document {doc_index} Summary:")
            for summary in result.summaries:
                # Wrap text to 120 characters for better readability
                wrapped_text = textwrap.fill(summary.text, width=120)
                print(wrapped_text)
                print()  # Add a blank line for better readability
        elif result.is_error is True:
            print(f"Document {doc_index} has an error with code '{result.error.code}' and message '{result.error.message}'")

In [7]:
def key_phrase_extraction(client, documents):
    """
    Extracts key phrases from the provided documents using the Azure Text Analytics client.

    Args:
        client (TextAnalyticsClient): An authenticated Azure Text Analytics client.
        documents (list of str): A list of text documents to analyze for key phrases.

    Returns:
        None: Prints the extracted key phrases for each document.
    """
    # Call the Azure Text Analytics API to extract key phrases from the documents
    result = client.extract_key_phrases(documents)

    print("Key Phrases:\n")

    # Iterate through the results for each document
    for idx, doc in enumerate(result):  # Enumerate to get the document index
        # Check if the result for the document is valid
        if not doc.is_error:
            # Print the key phrases for the document
            print("Key phrases in document #{}: {}".format(
                idx + 1,  # 1-based index for readability
                ", ".join(doc.key_phrases)  # Join the key phrases with commas
            ))
        else:
            # Print an error message if the document result contains an error
            print(f"Error in document with ID {doc.id}: {doc.error}")

In [8]:
def pii_recognition(client, documents):
    """
    Recognizes Personally Identifiable Information (PII) entities in the provided documents.

    Args:
        client (TextAnalyticsClient): The authenticated Azure Text Analytics client.
        documents (list): A list of documents to analyze.

    Returns:
        None
    """
    # Call the Azure Text Analytics API to recognize PII entities
    response = client.recognize_pii_entities(documents, language="en")
    
    # Filter out documents that encountered errors during processing
    result = [doc for doc in response if not doc.is_error]
    
    # Iterate over the successfully processed documents
    for doc_idx, doc in enumerate(result, start=1):  # Start document numbering from 1
        print(f"Document #{doc_idx}:")
        print(f"Redacted Text: {doc.redacted_text}")
        for entity in doc.entities:
            print(f"Entity: {entity.text}")
            print(f"\tCategory: {entity.category}")
            print(f"\tConfidence Score: {entity.confidence_score}")
            print(f"\tOffset: {entity.offset}")
            print(f"\tLength: {entity.length}")
        print("\n")  # Add a newline for better readability between documents

In [9]:
def analyze_healthcare_entities(client, documents):
    """
    Analyzes healthcare entities in the provided documents using Azure Text Analytics.

    Args:
        client (TextAnalyticsClient): The Azure Text Analytics client.
        documents (list of str): A list of documents to analyze.

    Returns:
        None: Prints the extracted healthcare entities and their details.
    """
    # Start the healthcare entity analysis process
    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()
    
    # Filter out documents that encountered errors
    docs = [doc for doc in result if not doc.is_error]

    # Iterate through each document in the results
    for i, doc in enumerate(docs):
        # Extract the document number from the original text (if available)
        document_text = documents[i]
        doc_number = document_text.split("|")[0].strip() if "RECORD #" in document_text else f"Document {i + 1}"
        print(f"\nProcessing {doc_number}:\n")
        
        # Iterate through each entity in the document
        for entity in doc.entities:
            print(f"Entity: {entity.text}")
            print(f"...Normalized Text: {entity.normalized_text}")
            print(f"...Category: {entity.category}")
            print(f"...Subcategory: {entity.subcategory}")
            print(f"...Offset: {entity.offset}")
            print(f"...Confidence score: {entity.confidence_score}")
            
            # Print data sources if available
            if entity.data_sources is not None:
                print("...Data Sources:")
                for data_source in entity.data_sources:
                    print(f"......Entity ID: {data_source.entity_id}")
                    print(f"......Name: {data_source.name}")
            
            # Print assertion details if available
            if entity.assertion is not None:
                print("...Assertion:")
                print(f"......Conditionality: {entity.assertion.conditionality}")
                print(f"......Certainty: {entity.assertion.certainty}")
                print(f"......Association: {entity.assertion.association}")
        
        # Print relationships between entities
        for relation in doc.entity_relations:
            print(f"Relation of type: {relation.relation_type} has the following roles")
            for role in relation.roles:
                print(f"...Role '{role.name}' with entity '{role.entity.text}'")

In [10]:
document1 = [
    """
    Title: Annual Report on Company Performance

    Introduction: The year 2024 has been a remarkable period for XYZ Corporation. Despite the global economic challenges, we have achieved significant milestones in our journey towards innovation and sustainability.

    Financial Overview: Our revenue for the fiscal year 2024 was $5.2 billion, marking a 10% increase from the previous year. The net profit stood at $1.1 billion, reflecting our efficient cost management and strategic investments.

    Key Achievements:

    Product Launches: We successfully launched three new products - the XYZ Smartwatch, XYZ Home Assistant, and XYZ Electric Scooter. These products have received positive feedback from customers and industry experts alike.
    Sustainability Initiatives: Our commitment to sustainability was reinforced by reducing our carbon footprint by 20% and increasing the use of renewable energy sources in our operations.
    Market Expansion: We expanded our market presence to five new countries, including Brazil, India, and South Africa, enhancing our global footprint.
    Customer Feedback: Customer satisfaction remains a top priority for us. We conducted surveys and received over 10,000 responses, with 85% of customers expressing satisfaction with our products and services. However, some customers highlighted areas for improvement, particularly in customer support and delivery times.

    Challenges: The primary challenges we faced included supply chain disruptions and fluctuating market demands. We are actively working on strategies to mitigate these issues and ensure seamless operations.

    Future Outlook: Looking ahead, we aim to continue our growth trajectory by focusing on innovation, customer-centric solutions, and sustainable practices. Our goal is to achieve a 15% revenue growth in the next fiscal year.

    Conclusion: We are grateful to our employees, partners, and customers for their unwavering support. Together, we will strive to reach new heights and make a positive impact on the world.

    Contact Information: For more details, please contact John Doe, Chief Financial Officer, at john.doe@xyzcorp.com or call (123) 456-7890.
    """
]

In [11]:
customer_feedback = [
    """
    Customer satisfaction remains a top priority for us. 
    We conducted surveys and received over 10,000 responses, with 85% of customers expressing satisfaction with our products and services. 
    However, some customers highlighted areas for improvement, particularly in customer support and delivery times.
    """
]

In [12]:
sentiment_analysis_with_opinion_mining(client, customer_feedback)

Document 1 Sentiment: positive
Overall scores: positive=0.72; neutral=0.28; negative=0.00
  Sentence 1:      Customer satisfaction remains a top priority for us.      
    Sentiment: positive
    Scores: Positive=0.57, Neutral=0.43, Negative=0.00
    Mined Opinion 1:
      Target 'Customer satisfaction' (positive)
      Scores: Positive=1.00, Negative=0.00
        Assessment 'top priority' (positive)
        Scores: Positive=1.00, Negative=0.00
  Sentence 2: We conducted surveys and received over 10,000 responses, with 85% of customers expressing satisfaction with our products and services.      
    Sentiment: positive
    Scores: Positive=0.87, Neutral=0.13, Negative=0.00
    Mined Opinion 1:
      Target 'products' (positive)
      Scores: Positive=1.00, Negative=0.00
        Assessment 'satisfaction' (positive)
        Scores: Positive=1.00, Negative=0.00
    Mined Opinion 2:
      Target 'services' (positive)
      Scores: Positive=1.00, Negative=0.00
        Assessment 'satisfact

In [13]:
entity_recognition(client, document1)

Named Entities:

Document 1:

	Text: 	 Annual 	Category: 	 DateTime 	SubCategory: 	 Set 
	Confidence Score: 	 1.0 	Length: 	 6 	Offset: 	 12 

	Text: 	 year 2024 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 0.94 	Length: 	 9 	Offset: 	 72 

	Text: 	 XYZ Corporation 	Category: 	 Organization 	SubCategory: 	 None 
	Confidence Score: 	 1.0 	Length: 	 15 	Offset: 	 115 

	Text: 	 innovation 	Category: 	 Skill 	SubCategory: 	 None 
	Confidence Score: 	 0.83 	Length: 	 10 	Offset: 	 235 

	Text: 	 fiscal year 2024 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 0.94 	Length: 	 16 	Offset: 	 311 

	Text: 	 $5.2 billion 	Category: 	 Quantity 	SubCategory: 	 Currency 
	Confidence Score: 	 1.0 	Length: 	 12 	Offset: 	 332 

	Text: 	 10% 	Category: 	 Quantity 	SubCategory: 	 Percentage 
	Confidence Score: 	 1.0 	Length: 	 3 	Offset: 	 356 

	Text: 	 previous year 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 1.0 	Length: 	 13 

In [14]:
key_phrase_extraction(client, document1)

Key Phrases:

Key phrases in document #1: efficient cost management, renewable energy sources, supply chain disruptions, five new countries, XYZ Home Assistant, XYZ Electric Scooter, Chief Financial Officer, fluctuating market demands, global economic challenges, next fiscal year, three new products, new heights, XYZ Corporation, Financial Overview, XYZ Smartwatch, global footprint, Market Expansion, market presence, Annual Report, Company Performance, remarkable period, significant milestones, previous year, net profit, strategic investments, Key Achievements, Product Launches, positive feedback, industry experts, carbon footprint, South Africa, Customer Feedback, top priority, customer support, delivery times, primary challenges, Future Outlook, growth trajectory, customer-centric solutions, sustainable practices, unwavering support, positive impact, Contact Information, Customer satisfaction, seamless operations, 15% revenue growth, Sustainability Initiatives, John Doe, Title, Intro

In [15]:
extractive_summarization(client, document1)
abstractive_summarization(client, document1)

Document 1 Summary:
  Sentence 1: Title: Annual Report on Company Performance
  Sentence 2: Introduction: The year 2024 has been a remarkable period for XYZ Corporation.
  Sentence 3: Despite the global economic challenges, we have achieved significant milestones in our journey towards innovation and sustainability.
  Sentence 4: Financial Overview: Our revenue for the fiscal year 2024 was $5.2 billion, marking a 10% increase from the previous year.
  Sentence 5: Product Launches: We successfully launched three new products - the XYZ Smartwatch, XYZ Home Assistant, and XYZ Electric Scooter.
Document 1 Summary:
XYZ Corporation has demonstrated resilience and growth in 2024, achieving a revenue of $5.2 billion with a 10% increase
from the previous year and a net profit of $1.1 billion, despite global economic challenges. The company has
successfully launched three innovative products and expanded its global footprint by entering five new markets,
including Brazil, India, and South Africa

In [16]:
pii_recognition(client, document1)

Document #1:
Redacted Text: 
    Title: Annual Report on Company Performance

    Introduction: The year 2024 has been a remarkable period for ***************. Despite the global economic challenges, we have achieved significant milestones in our journey towards innovation and sustainability.

    Financial Overview: Our revenue for the fiscal year 2024 was $5.2 billion, marking a 10% increase from the previous year. The net profit stood at $1.1 billion, reflecting our efficient cost management and strategic investments.

    Key Achievements:

    Product Launches: We successfully launched three new products - the XYZ Smartwatch, XYZ Home Assistant, and XYZ Electric Scooter. These products have received positive feedback from ********* and **************** alike.
    Sustainability Initiatives: Our commitment to sustainability was reinforced by reducing our carbon footprint by 20% and increasing the use of renewable energy sources in our operations.
    Market Expansion: We expanded o

In [17]:
document2 = [
    """
    Patient Information:

    Name: Jane Doe
    Date of Birth: March 15, 1985
    Gender: Female
    Medical Record Number: 123456789

    Visit Information:

    Date of Visit: April 10, 2025
    Physician: Dr. John Smith
    Department: Cardiology
    Chief Complaint: Jane Doe presented with complaints of chest pain and shortness of breath that started two days ago. She describes the pain as sharp and localized to the left side of her chest.

    History of Present Illness: The patient reports that the chest pain began suddenly while she was at rest. She denies any recent physical exertion or trauma. The pain is exacerbated by deep breathing and relieved by sitting up. She also reports episodes of dizziness and palpitations.

    Past Medical History:

    Hypertension
    Hyperlipidemia
    Asthma

    Medications:

    Lisinopril 10 mg daily
    Atorvastatin 20 mg daily
    Albuterol inhaler as needed
    Allergies:

    Penicillin

    Family History:

    Father: Hypertension, myocardial infarction at age 60
    Mother: Type 2 diabetes

    Social History:

    Non-smoker
    Occasional alcohol consumption
    Works as a software engineer

    Review of Systems:

    Cardiovascular: Positive for chest pain and palpitations
    Respiratory: Positive for shortness of breath, negative for cough
    Gastrointestinal: Negative for nausea, vomiting, or abdominal pain
    Neurological: Positive for dizziness, negative for headaches or seizures

    Physical Examination:

    Vital Signs: Blood pressure 150/90 mmHg, heart rate 95 bpm, respiratory rate 18 breaths per minute, temperature 98.6°F
    General: Alert and oriented, no acute distress
    Cardiovascular: Regular rhythm, no murmurs, rubs, or gallops
    Respiratory: Clear to auscultation bilaterally, no wheezing or crackles
    Abdomen: Soft, non-tender, no organomegaly
    Neurological: Cranial nerves intact, no focal deficits
    Diagnostic Tests:

    Electrocardiogram (ECG): Normal sinus rhythm, no ST-segment changes
    Chest X-ray: No acute cardiopulmonary abnormalities
    Blood Tests: Elevated troponin levels, normal complete blood count (CBC), normal electrolytes
    Assessment: The patient is diagnosed with acute coronary syndrome (ACS) based on her symptoms and elevated troponin levels.

    Plan:

    Admit to the hospital for further monitoring and management.
    Initiate aspirin 81 mg daily and clopidogrel 75 mg daily.
    Continue current medications for hypertension and hyperlipidemia.
    Consult cardiology for possible coronary angiography.
    Educate the patient on lifestyle modifications and the importance of medication adherence.
    Follow-Up: The patient will be re-evaluated in 24 hours and monitored for any changes in her condition.

    Signature: Dr. John Smith, MD
    """
]

In [18]:
entity_recognition(client, document2)

Named Entities:

Document 1:

	Text: 	 Patient Information 	Category: 	 Skill 	SubCategory: 	 None 
	Confidence Score: 	 0.94 	Length: 	 19 	Offset: 	 5 

	Text: 	 Jane Doe 	Category: 	 Person 	SubCategory: 	 None 
	Confidence Score: 	 0.98 	Length: 	 8 	Offset: 	 37 

	Text: 	 March 15, 1985 	Category: 	 DateTime 	SubCategory: 	 Date 
	Confidence Score: 	 1.0 	Length: 	 14 	Offset: 	 65 

	Text: 	 Female 	Category: 	 PersonType 	SubCategory: 	 None 
	Confidence Score: 	 0.53 	Length: 	 6 	Offset: 	 92 

	Text: 	 123456789 	Category: 	 PhoneNumber 	SubCategory: 	 None 
	Confidence Score: 	 0.8 	Length: 	 9 	Offset: 	 126 

	Text: 	 April 10, 2025 	Category: 	 DateTime 	SubCategory: 	 Date 
	Confidence Score: 	 1.0 	Length: 	 14 	Offset: 	 180 

	Text: 	 Physician 	Category: 	 PersonType 	SubCategory: 	 None 
	Confidence Score: 	 0.99 	Length: 	 9 	Offset: 	 199 

	Text: 	 Dr 	Category: 	 PersonType 	SubCategory: 	 None 
	Confidence Score: 	 0.57 	Length: 	 2 	Offset: 	 210 

	Text: 	 J

In [19]:
key_phrase_extraction(client, document2)

Key Phrases:

Key phrases in document #1: Female    Medical Record Number, focal deficits    Diagnostic Tests, Dr. John Smith    Department, normal complete blood count, recent physical exertion, Occasional alcohol consumption, software engineer    Review, seizures    Physical Examination, acute cardiopulmonary abnormalities, Elevated troponin levels, acute coronary syndrome, possible coronary angiography, Past Medical History, Normal sinus rhythm, Penicillin    Family History, acute distress    Cardiovascular, Jane Doe    Date, Blood Tests, normal electrolytes, Blood pressure, Regular rhythm, Social History, Chief Complaint, left side, Present Illness, deep breathing, Asthma    Medications, Albuterol inhaler, myocardial infarction, Type 2 diabetes, cough    Gastrointestinal, Vital Signs, heart rate, Cranial nerves, current medications, lifestyle modifications, medication adherence, Chest X-ray, abdominal pain, Visit Information, ST-segment changes, chest pain, respiratory rate, Patien

In [20]:
extractive_summarization(client, document2)
abstractive_summarization(client, document2)

Document 1 Summary:
  Sentence 1: Chief Complaint: Jane Doe presented with complaints of chest pain and shortness of breath that started two days ago.
  Sentence 2: She describes the pain as sharp and localized to the left side of her chest.
  Sentence 3: She also reports episodes of dizziness and palpitations.
  Sentence 4: Cardiovascular: Positive for chest pain and palpitations
  Sentence 5: Assessment: The patient is diagnosed with acute coronary syndrome (ACS) based on her symptoms and elevated troponin levels.
Document 1 Summary:
Jane Doe, a 39-year-old female with a history of hypertension, hyperlipidemia, and asthma, visited the cardiology
department with chief complaints of chest pain localized to the left side, shortness of breath, dizziness, and
palpitations. Despite no recent physical exertion or trauma, her symptoms suggested a cardiac event. Tests including an
ECG and chest X-ray showed no immediate abnormalities, but elevated troponin levels indicated a possible acute co

In [22]:
analyze_healthcare_entities(client, document2)


Processing Document 1:

Entity: Birth
...Normalized Text: Birth
...Category: Diagnosis
...Subcategory: None
...Offset: 58
...Confidence score: 0.66
...Data Sources:
......Entity ID: C0005615
......Name: UMLS
......Entity ID: 0000001722
......Name: AOD
......Entity ID: 0000001931
......Name: CHV
......Entity ID: 2401-7479
......Name: CSP
......Entity ID: GO:0007567
......Name: GO
......Entity ID: sh85098407
......Name: LCH_NW
......Entity ID: D036801
......Name: MSH
......Entity ID: 06050
......Name: PSY
......Entity ID: Xa4eI
......Name: RCD
......Entity ID: F-31540
......Name: SNM
......Entity ID: F-88005
......Name: SNMI
......Entity ID: 3950001
......Name: SNOMEDCT_US
Entity: March 15, 1985
...Normalized Text: None
...Category: Date
...Subcategory: None
...Offset: 65
...Confidence score: 1.0
Entity: Female
...Normalized Text: Woman
...Category: Gender
...Subcategory: None
...Offset: 92
...Confidence score: 1.0
...Data Sources:
......Entity ID: C0043210
......Name: UMLS
......Entity